In [30]:
import re
import pandas as pd

from datetime import datetime, timedelta

In [31]:
df_vagas = pd.read_excel('data/data_raw/vagas_vagas_raw.xlsx')
df_vagas.head()

,site_da_vaga,link_site,link_origem,data_publicacao,data_expiracao_vaga,data_coleta,posicao,senioridade,titulo_da_vaga,nome_da_empresa,cidade,estado,modalidade,contrato,regime,beneficios,codigo_vaga,descricao
0,Vagas.com,https://www.vagas.com.br/vagas/v2597008/analis...,NaN,Publicada em 29/12/2023,NaN,2024-02-03,Analista de Dados,Pleno,Analista de Dados,Tangará,NaN,Minas Gerais,NaN,Não informado,Regime CLT,"['Assistência médica', 'Assistência odontológi...",v2597008,Descrição\nDescrição: Atividades:\nRealizar at...
1,Vagas.com,https://www.vagas.com.br/vagas/v2590354/analis...,NaN,Publicada em 07/12/2023,2024-02-29,2024-02-03,Analista de Dados,Pleno,Analista de Dados Pleno,DHL Express,São Paulo,SP,NaN,Não informado,Regime CLT,"['Assistência médica', 'Assistência odontológi...",v2590354,Descrição\nDescrição: SUAS ATIVIDADES:\nColeta...
2,Vagas.com,https://www.vagas.com.br/vagas/v2604780/analis...,NaN,Publicada em 23/01/2024,2024-02-23,2024-02-03,Analista de Dados,Júnior/Trainee,Analista de Dados Júnior,JCDecaux do Brasil,São Paulo,SP,NaN,Não informado,Regime CLT,"['Assistência médica', 'Assistência odontológi...",v2604780,Descrição\nDescrição: ▪ Minerar os dados e pro...
3,Vagas.com,https://www.vagas.com.br/vagas/v2607407/analis...,NaN,Publicada há 2 dias,2024-02-07,2024-02-03,Analista de Dados,Pleno,Analista de Dados e Pesquisa Pleno - (Ênfase e...,Fundação Faculdade de Medicina,São Paulo,SP,NaN,Não informado,Regime CLT,NaN,v2607407,Descrição\nÁrea de Atuação: FFM - Saúde Digita...
4,Vagas.com,https://www.vagas.com.br/vagas/v2599242/analis...,NaN,Publicada em 08/01/2024,2024-02-08,2024-02-03,Analista de Dados,Sênior,Analista de Dados de Survey,Oceaneering,Macaé,RJ,NaN,Não informado,Regime CLT,"['Assistência médica', 'Assistência odontológi...",v2599242,Descrição\nDescrição: Resumo:O analista de dad...


In [32]:
df_vagas['regime'].value_counts()

regime
Regime CLT         554
Estágio             64
Temporário          46
Pessoa Jurídica      8
Name: count, dtype: int64

In [33]:
def clean_publish_date(date_string: str) -> str | int:

    if 'dias' in date_string:
        days = re.findall(r'\d+', date_string)[0]
        return int(days)
    
    elif 'ontem' in date_string:
        return 1
    
    elif 'hoje' in date_string:
        return 0
    
    else:

        date = date_string.split(' ')[-1]

        date = date.split('/')
        date.reverse()

        formatted_date = '-'.join(date)

        return formatted_date


def subtract_date(date: str, days: int) -> str:
    
    real_date_datetime = datetime.strptime(date, '%Y-%m-%d') - timedelta(days)
    real_date_string = real_date_datetime.strftime('%Y-%m-%d')

    return real_date_string

In [34]:
state_uf = {
            'Acre': 'AC',
            'Alagoas': 'AL',
            'Amazonas': 'AM',
            'Amapá': 'AP',
            'Bahia': 'BA',
            'Ceará': 'CE',
            'Distrito Federal': 'DF',
            'Espírito Santo': 'ES',
            'Goiás': 'GO',
            'Maranhão': 'MA',
            'Mato Grosso': 'MT',
            'Mato Grosso do Sul': 'MS',
            'Minas Gerais': 'MG',
            'Pará': 'PA',
            'Paraíba': 'PB',
            'Paraná': 'PR',
            'Pernambuco': 'PE',
            'Piauí': 'PI',
            'Rio de Janeiro': 'RJ',
            'Rio Grande do Norte': 'RN',
            'Rio Grande do Sul': 'RS',
            'Rondônia': 'RO',
            'Roraima': 'RR',
            'Santa Catarina': 'SC',
            'São Paulo': 'SP',
            'Sergipe': 'SE',
            'Tocantins': 'TO'
    }


def get_state_uf(state: str) -> str:
    
    if state_uf.get(state) is None:
        return state
    else:
        return state_uf.get(state)

In [35]:
def search_description_regex(search_words: list[str], job_description: str) -> list[str]:
    
    search_mask = rf'({"|".join(search_words)})'

    matches = re.findall(search_mask, job_description, flags=re.I)

    matches = list( map( str.lower, matches ) )

    return matches


def fill_modality_column(search_words: list[str], job_description: str) -> str:

    matched_words = search_description_regex(search_words, job_description)
    
    search_words = search_words[:]      # cria uma copia da lista (não afeta a variável global passada como referência)
    search_words.remove('Presencial')

    if 'híbrido' in matched_words:
        return 'Híbrido'
    
    elif any( True if word.lower() in matched_words else False for word in search_words):
        return 'Remoto'
    
    else:
        return 'Presencial'
    

def fill_pcd_column(search_words: list[str], job_description: str) -> str:
    
    matched_words = search_description_regex(search_words, job_description)

    search_condition = any( True if pcd_word.lower() in matched_words else False for pcd_word in search_words )

    if search_condition:
        return 'Sim'
    else:
        return 'Não informado'

In [36]:
df_vagas['data_publicacao'] = df_vagas['data_publicacao'].apply( lambda date: clean_publish_date(date) )

df_vagas['estado'] = df_vagas['estado'].apply( lambda state: get_state_uf(state) )

df_vagas['contrato'] = df_vagas['regime'].apply( lambda regime: 'Efetivo'    if regime == 'Regime CLT'      else
                                                                'Associado'  if regime == 'Pessoa Jurídica' else
                                                                'Estágio'    if regime == 'Estágio'         else
                                                                'Temporário' if regime == 'Temporário'      else
                                                                'Não informado'
                                                    )


df_vagas['data_publicacao'] = df_vagas.apply( 
                                        lambda x: 
                                             subtract_date(x['data_coleta'], x['data_publicacao']) 
                                             if isinstance(x['data_publicacao'], int) else 
                                             x['data_publicacao'], axis=1 
                                        )



modalidade_search = ['Híbrido', 'Remoto', 'Home Office', 'Presencial']

df_vagas['modalidade'] = df_vagas.apply( lambda columns: 'Remoto' if columns['estado'] == 'Todo o Brasil' else 
                                                          fill_modality_column(modalidade_search, columns['descricao']), 
                                        axis=1
                                 )


pcd_search = ['PcD', 'deficiência', 'deficiente']

df_vagas['pcd'] = df_vagas['descricao'].apply( lambda description: fill_pcd_column(pcd_search, description) )

In [37]:
df_vagas

,site_da_vaga,link_site,link_origem,data_publicacao,data_expiracao_vaga,data_coleta,posicao,senioridade,titulo_da_vaga,nome_da_empresa,cidade,estado,modalidade,contrato,regime,beneficios,codigo_vaga,descricao,pcd
0,Vagas.com,https://www.vagas.com.br/vagas/v2597008/analis...,NaN,2023-12-29,NaN,2024-02-03,Analista de Dados,Pleno,Analista de Dados,Tangará,NaN,MG,Presencial,Efetivo,Regime CLT,"['Assistência médica', 'Assistência odontológi...",v2597008,Descrição\nDescrição: Atividades:\nRealizar at...,Não informado
1,Vagas.com,https://www.vagas.com.br/vagas/v2590354/analis...,NaN,2023-12-07,2024-02-29,2024-02-03,Analista de Dados,Pleno,Analista de Dados Pleno,DHL Express,São Paulo,SP,Presencial,Efetivo,Regime CLT,"['Assistência médica', 'Assistência odontológi...",v2590354,Descrição\nDescrição: SUAS ATIVIDADES:\nColeta...,Não informado
2,Vagas.com,https://www.vagas.com.br/vagas/v2604780/analis...,NaN,2024-01-23,2024-02-23,2024-02-03,Analista de Dados,Júnior/Trainee,Analista de Dados Júnior,JCDecaux do Brasil,São Paulo,SP,Presencial,Efetivo,Regime CLT,"['Assistência médica', 'Assistência odontológi...",v2604780,Descrição\nDescrição: ▪ Minerar os dados e pro...,Não informado
3,Vagas.com,https://www.vagas.com.br/vagas/v2607407/analis...,NaN,2024-02-01,2024-02-07,2024-02-03,Analista de Dados,Pleno,Analista de Dados e Pesquisa Pleno - (Ênfase e...,Fundação Faculdade de Medicina,São Paulo,SP,Presencial,Efetivo,Regime CLT,NaN,v2607407,Descrição\nÁrea de Atuação: FFM - Saúde Digita...,Não informado
4,Vagas.com,https://www.vagas.com.br/vagas/v2599242/analis...,NaN,2024-01-08,2024-02-08,2024-02-03,Analista de Dados,Sênior,Analista de Dados de Survey,Oceaneering,Macaé,RJ,Híbrido,Efetivo,Regime CLT,"['Assistência médica', 'Assistência odontológi...",v2599242,Descrição\nDescrição: Resumo:O analista de dad...,Sim
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
667,Vagas.com,https://www.vagas.com.br/vagas/v2604403/consul...,NaN,2024-01-22,2024-02-20,2024-02-03,Engenharia de Dados,Sênior,Consultor(a) de Riscos Financeiros (Vaga Exclu...,Vivo (Telefônica Brasil),São Paulo,SP,Presencial,Efetivo,Regime CLT,"['Assistência médica', 'Assistência odontológi...",v2604403,Descrição\nDescrição: Vaga Exclusiva para Prof...,Sim
668,Vagas.com,https://www.vagas.com.br/vagas/v2591690/superv...,NaN,2023-12-12,2024-02-29,2024-02-03,Engenharia de Dados,Supervisão/Coordenação,Supervisor de Produção,DFL Indústria e Comércio S/A,Rio de Janeiro,RJ,Presencial,Efetivo,Regime CLT,"['Assistência médica', 'Assistência odontológi...",v2591690,Descrição\nDescrição: \nControlar e orientar a...,Não informado
669,Vagas.com,https://www.vagas.com.br/vagas/v2604407/especi...,NaN,2024-01-22,2024-02-20,2024-02-03,Engenharia de Dados,Sênior,Especialista Inovação e Desenvolvimento de Nov...,Vivo (Telefônica Brasil),São Paulo,SP,Presencial,Efetivo,Regime CLT,"['Assistência médica', 'Assistência odontológi...",v2604407,Descrição\nDescrição: Vaga Exclusiva para Prof...,Sim
670,Vagas.com,https://www.vagas.com.br/vagas/v2604177/consul...,NaN,2024-01-22,2024-02-20,2024-02-03,Engenharia de Dados,Pleno,Consultor(a) de Transformação Digital (Vaga Ex...,Vivo (Telefônica Brasil),São Paulo,SP,Presencial,Efetivo,Regime CLT,"['Assistência médica', 'Assistência odontológi...",v2604177,Descrição\nDescrição: Vaga exclusive para prof...,Sim


In [38]:
df_vagas['modalidade'].value_counts()

modalidade
Presencial    567
Remoto         55
Híbrido        50
Name: count, dtype: int64

In [39]:
df_vagas['pcd'].value_counts()

pcd
Não informado    588
Sim               84
Name: count, dtype: int64